In [1]:
%reset -f
from django_for_jupyter import init_django
init_django()

/home/javier/hd1/Codigo/JAVIER/test1/comparaprecios-1


[nltk_data] Downloading package stopwords to /home/javier/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/javier/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
from precios.models import (Site, 
                            Marcas, 
                            Articulos, 
                            Vendedores, 
                            Unifica, 
                            Settings, 
                            SiteURLResults,
                            TaggedArticles,
                            )

from taggit.models import  Tag
from precios.pi_get import (
    reemplaza_palabras,
    get_marcas_que_me_apuntan
)
from precios.pi_rules import (
    intenta_marca,
)   
from django.db.models import Count

In [ ]:
sites_ean = Site.objects.filter(es_ean13=True).values_list('id', flat=True)
ids_ean = SiteURLResults.objects.filter(site__id__in=sites_ean).values('idproducto').annotate(num_marcas=Count('marca', distinct=True)).order_by('num_marcas').filter(num_marcas__gt=1)
# ids_ean = ids_ean.filter(idproducto='9002490100070')
print(len(ids_ean))

In [ ]:
def is_marca(marca_str):
    if Marcas.objects.filter(nombre=marca_str, es_marca=True).exists():
        la_marca = Marcas.objects.filter(nombre=marca_str, es_marca=True).get()
        
        return 'habilitada', la_marca.resultsCount
    if Marcas.objects.filter(nombre=marca_str, es_marca=False).exists():
        la_marca = Marcas.objects.filter(nombre=marca_str, es_marca=False).get()
        # get_marcas_que_me_apuntan(la_marca)
        return 'des-habilitada', la_marca.resultsCount
    
    return 'no existe', 0

In [ ]:
# ============================
# siteurlresults
for id_ean in ids_ean[1320:]:

    marcas = SiteURLResults.objects.filter(idproducto=id_ean['idproducto']).values('marca').annotate(num_marcas2=Count('marca')).order_by('marca')
    # .filter(num_marcas__gt=1)
    print('=====================================')
    
    print(id_ean['idproducto'], 'numero de marcas=',id_ean['num_marcas'])

    todas_habilitadas       = True
    todas_deshabilitadas    = True
    diferentes              = False
    maximo_marca            = 0
    que_marca               = ''
    for marca in marcas:
        es_marca, cuantos = is_marca(marca['marca'])
        if es_marca == 'habilitada' :
            todas_deshabilitadas = False
        if es_marca == 'des-habilitada' :
            todas_habilitadas = False
        if es_marca == 'no existe':
            todas_habilitadas = False

        if cuantos > maximo_marca:
            maximo_marca = cuantos
            que_marca    = marca['marca']

        cuales = SiteURLResults.objects.filter(idproducto=id_ean['idproducto'], marca=marca['marca'])
        if es_marca == 'habilitada' :
            print(marca, es_marca, cuantos)
        # for cual in cuales:
        #     print(cual.nombre)

        # print('---------------------------------')

    if todas_habilitadas:
        print(f'Todas habilitadas, necesita regla por marca hacia: |{que_marca}|')
    if todas_deshabilitadas:
        print('Todas Deshabilitadas, nada que hacer')
